#afw.Image.ExposureF('file.fits.fz[i]') returns the image in 'file.fits.fz[1]'

* [DM-2599](https://jira.lsstcorp.org/browse/DM-2599), John Swinbank, 3 SPs.

Issue description:

> It seems that afwImage.ExposureF ignores the extension number when this is passed on as part of the filename and uses the image in extension number 1. This is not the case with afwImage.MaskedImageF which correctly uses the input extension number passed in the same way.
> The problem has been checked on OSX Yosemite 10.10.3 with 
> the is illustrated in the following code https://gist.github.com/anonymous/d10c4a79d94c1393a493
> which also requires the following image in the working directory:
> http://www.astro.washington.edu/users/krughoff/data/c4d_130830_040651_ooi_g_d1.fits.fz
 
This notebook has been checked to run against the release tagged `w_2015_26`.

Note that at time of writing the data file referred to in the issue description (`c4d_...fits.fz`) was available at the location mentioned and is used in the examples below. However, it's quite big (~300 MB) and is not included with this notebook. 

The issue here is broader than the description above leads one to suppose. In fact, this is a confusing result of our attempt to shoe-horn a rich data model into the FITS format.

Both masked images (`lsst.afw.image.MaskedImageF`, etc) and exposures (`lsst.afw.image.ExposureF`, etc) provide an image with associated mask and variance planes. When serialized to FITS, they are stored as successive extensions in a multi-extension FITS file (MEF). See [LSST Doxygen](https://lsst-web.ncsa.illinois.edu/doxygen/x_masterDoxyDoc/afw_sec_image_i_o.html) for details.

This works well enough when the code is simply used to re-ingest an image which has been produced by the LSST stack. However, when the user tries to load a arbitrary multi-extension FITS file using our code, the unexpected can happen. For example, it's easy to end up with "image" N+2 in fact be the variance of image N:

In [1]:
import lsst.afw.image as afwImage
im1 = afwImage.MaskedImageF("c4d_130830_040651_ooi_g_d1.fits.fz[1]")
im3 = afwImage.MaskedImageF("c4d_130830_040651_ooi_g_d1.fits.fz[3]")
assert(im1.get(0,0)[2] == im3.get(0,0)[0])

In fact, as [Jim Bosch points out](https://jira.lsstcorp.org/browse/DM-2599?focusedCommentId=34654&page=com.atlassian.jira.plugin.system.issuetabpanels:comment-tabpanel#comment-34654), the Exposure data model is complex, and potentially even more confusing things than the above can happen.

Defining new or more comprehensive data models is out of scope for this work. Rather, our ultimate aim here is to help the user by providing appropriate warnings or throwing errors whenever they are doing something that will likely result in incorrect or confusing results.

The starting point for this was [suggested by Simon Krughoff](https://jira.lsstcorp.org/browse/DM-2599?focusedCommentId=34658&page=com.atlassian.jira.plugin.system.issuetabpanels:comment-tabpanel#comment-34658), but there are a bunch of corner cases which are extensively covered in the [unit tests](https://github.com/lsst/afw/blob/master/tests/maskedImageIO.py#L191). Specifically, we attempt to make this as safe as possible by handling the following situations. Note that Exposures always set `needAllHdus` to `false`.

* If `needAllHdus` is `true`:
  * If the user has specified a non-default HDU, we throw.
  * If the user has not specified an HDU (or has specified the default):
    * If any of the image, mask or variance is unreadable, we throw.
    * Otherwise, return the MaskedImage with image/mask/variance as expected.
* If `needAllHdus` is `false`:
  * If the user has specified a non-default HDU:
    * If the user specified HDU is unreadable, we throw.
    * Otherwise, we return the contents of that HDU as the image and default (=empty) mask & variance.
  * If the user has not specified an HDU, or has specified one equal to the default:
    * If the default HDU is unreadable, we throw.
    * Otherwise, we attempt to read both mask and variance from the FITS file, and return them together with the image. If one or both are unreadable, we fall back to an empty default for the missing data and return the remainder.

Further: when we write data to a MEF, we add an `EXTTYPE` header to indicate the type of data being written (`IMAGE`, `MASK` or `VARIANCE`). This header is checked when the data is read, and an error thrown if the data being read is of the wrong type. However, it was decided that it was not possible simply to throw an error if the header is missing: the header as not been consistently added in all past versions of the code, and it would cause too much backwards incompatibility. However, to help the user, we log warnings if the expected header isn't found.

For the purposes of this demo, we're going to want to look at log output from the LSST stack. Unfortunately, there's not an obvious way to cleanly include this in an IPython notebook (suggestions welcome...). As a workaround, we'll define a context manager as follows:

In [2]:
from __future__ import print_function
from contextlib import contextmanager
from tempfile import NamedTemporaryFile
import lsst.pex.logging as logging

@contextmanager
def capture_log():
    tf = NamedTemporaryFile()
    root = logging.getDefaultLog()
    root.addDestination(logging.FileDestination(tf.name, logging.BriefFormatter()))
    yield
    print(tf.read())

Going back to the original FITS file which caused the problem, what does this mean in practice? Well, if we load a MaskedImage without specifying an HDU, we are in the final regime in the list above: we attempt to read three successive extensions, starting at the default (first) and providing empty values if the data cannot be read. Furthermore, we log messages if the `EXTTYPE` header is not found.

In [3]:
with capture_log():
    im = afwImage.MaskedImageF("c4d_130830_040651_ooi_g_d1.fits.fz")
im.get(0,0) # Print image, mask, variance at given pixel as tuple

afw.image.MaskedImage WARNING: Expected extension type not found: IMAGE
afw.image.MaskedImage WARNING: Mask unreadable; using default
afw.image.MaskedImage WARNING: Expected extension type not found: VARIANCE



(97.41546630859375, 0, 80.07608795166016)

If we specify a particular extension, then (as above) "we return the contents of that HDU as the image and default (=empty) mask & variance". Plus we'll log a warning if that HDU doesn't an appropriate `EXTTYPE`.

In [4]:
with capture_log():
    im = afwImage.MaskedImageF("c4d_130830_040651_ooi_g_d1.fits.fz[2]")
im.get(0,0)

afw.image.MaskedImage WARNING: Expected extension type not found: IMAGE



(85.97016906738281, 0, 0.0)

Note that we got default (zero) values for both the mask and the variance above.

What about the cases where we throw exceptions? We'll need to jump through some hoops to create an appropriate image first.

In [5]:
import os
import pyfits
import numpy
import shutil
from tempfile import mkdtemp
import lsst.afw.geom as afwGeom

@contextmanager
def tempfits():
    hdus = pyfits.HDUList([pyfits.PrimaryHDU(numpy.array([[1]])),
                       pyfits.hdu.ImageHDU(None),
                       pyfits.hdu.ImageHDU(numpy.array([[2]]))])
    tempdir = mkdtemp()
    filename = os.path.join(tempdir, "test.fits")
    hdus.writeto(filename)
    try:
        yield filename
    finally:
        shutil.rmtree(tempdir, ignore_errors=True)

The above will generate a 3 extension FITS file, with the second extension being unreadable for our purposes. Let's consider an attempt to load an image from the default HDU:

In [6]:
needAllHdus = False

with tempfits() as filename, capture_log():
    im = afwImage.MaskedImageF(filename, None, afwGeom.Box2I(), afwImage.PARENT, False, needAllHdus)
im.get(0,0)

afw.image.MaskedImage WARNING: Expected extension type not found: IMAGE



(1.0, 0, 0.0)

So we get the image, log a warning, and default values for mask & variance (as above).

What about trying to load the unreadable extension as an image? That should throw...

In [7]:
needAllHdus = False

with tempfits() as filename, capture_log():
    im = afwImage.MaskedImageF(filename+"[1]", None, afwGeom.Box2I(), afwImage.PARENT, False, needAllHdus)

FitsError: 
  File "src/fits.cc", line 999, in void lsst::afw::fits::Fits::readImageImpl(int, T *, long *, long *, long *) [T = unsigned char]
    cfitsio error (/var/folders/jp/lqz3n0m17nqft7bwtw3b8n380000gp/T/tmpJrPNF4/test.fits[1]): illegal number of dimensions (320) : Reading image {0}
lsst::afw::fits::FitsError: 'cfitsio error (/var/folders/jp/lqz3n0m17nqft7bwtw3b8n380000gp/T/tmpJrPNF4/test.fits[1]): illegal number of dimensions (320) : Reading image'


...and it does.

The [unit tests](https://github.com/lsst/afw/blob/master/tests/maskedImageIO.py#L191) use a variation on that image construction code to demonstrate that we handle every case itemized in the list. I think demonstrating every case here is not a useful exercise.